# Running the model

Setup a `feisty` integration.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import yaml

import feisty

## Configure testcase to run

TODO: this is using the first column (which I believe is shallowest); want to add deepest column as well.
That will require refactoring the yaml file and also changing test_ds.

In [3]:
def _comparison(matlab_vals, py_vals, row_name, full_table, seps, thres):
    nstep = np.min([len(matlab_vals["time"].data), len(py_vals["time"].data)])
    nx = np.min([len(matlab_vals["X"].data), len(py_vals["X"].data)])
    matlab_vals = matlab_vals.isel(time=slice(0, nstep), X=slice(0, nx)).data
    py_vals = py_vals.isel(time=slice(0, nstep), X=slice(0, nx)).data
    if thres:
        thres_mask = np.logical_and(np.abs(py_vals) > thres, np.abs(matlab_vals) > thres)
        py_vals = np.where(thres_mask, py_vals, 0)
        matlab_vals = np.where(thres_mask, matlab_vals, 0)
    rel_err_denom = np.where(
        matlab_vals != 0, matlab_vals, 1
    )  # avoid dividing by 0 in next np.where() statement
    if np.any(np.isnan(py_vals)):
        print(f"python has nans for {rowname}!")
        return
    rel_errs = np.where(matlab_vals != 0, np.abs((py_vals - matlab_vals) / rel_err_denom), np.nan)
    rel_errs = np.where(np.logical_and(np.isnan(rel_errs), py_vals == 0), 0, rel_errs)
    rel_errs = np.where(np.logical_and(np.isnan(rel_errs), py_vals > 0), np.inf, rel_errs)
    rel_errs = np.where(np.logical_and(np.isnan(rel_errs), py_vals < 0), -np.inf, rel_errs)

    if full_table or rel_err > 1e-12:
        max_inds = np.nonzero(np.abs(rel_errs) == np.max(np.abs(rel_errs)))
        t_ind = max_inds[0][0]
        x_ind = max_inds[1][0]
        matlab_val = matlab_vals[t_ind][x_ind]
        py_val = py_vals[t_ind][x_ind]
        rel_err = rel_errs[t_ind][x_ind]
        print(
            f"{seps[0]}{row_name} (t={t_ind}, X={x_ind}){seps[1]}{matlab_val:10.4e}{seps[2]}"
            + f"{py_val:10.4e}{seps[2]}{rel_err:10.4e}{seps[-1]}"
        )


def compare_nc(baseline_ds, test_da_or_ds, full_table=True, markdown_formatting="true", thres=None):
    if type(test_da_or_ds) == xr.Dataset:
        ds = test_da_or_ds
        da = None
        baseline_da = None
        table_header = ''
    else:
        da = test_da_or_ds
        ds = None
        da_dims = ["group", "fish"]
        da_dim = ''
        for dad in da_dims:
            if dad in da.dims:
                da_dim = dad
                break
        baseline_da = baseline_ds[da.name]
        table_header = da_dim
    if markdown_formatting:
        print(f"| {table_header} | Matlab Value | Python Value | Rel Err |")
        print("| --- | --- | --- | --- |")
        seps = ["| ", " | ", " | ", " |"]
    else:
        seps = ["", ": ", ", ", ""]
    if da is not None:
        for n, dimname in enumerate(da[da_dim]):
            _comparison(
                baseline_da.isel({da_dim: n}),
                da.isel({da_dim: n}),
                dimname.data,
                full_table,
                seps,
                thres,
            )
    else:
        for varname in ds:
            _comparison(
                baseline_ds.isel(zooplankton=0)[varname],
                ds.isel(zooplankton=0)[varname],
                varname,
                full_table,
                seps,
                thres,
            )

In [4]:
# Configuration settings for the offline driver as well as output generated in this notebook

driver_config = dict()
for matlab_script in ['test_case', 'test_locs3', 'FOSI_cesm', 'FOSI_cesm_daily', 'FOSI_spinup']:
    driver_config[matlab_script] = dict()
    # default settings
    driver_config[matlab_script][
        'compare_forcing'
    ] = True  # Generate table to compare forcing to matlab
    driver_config[matlab_script][
        'force_nonnegative'
    ] = True  # When computing table to compare forcing to matlab, replace negative values of poc_flux_bottom, zooC, and zooC_mort with 0
    driver_config[matlab_script][
        'start_date'
    ] = '0001-01-01'  # Model date at beginning of simulation
    driver_config[matlab_script]['nyears'] = 1  # Length of run (in years)
    driver_config[matlab_script][
        'ignore_year_in_forcing'
    ] = False  # Cycle the forcing (assumes a single year of forcing is provided)
    driver_config[matlab_script][
        'settings_in'
    ] = {}  # Override FEISTY settings from feisty/core/default_settings.yml
    driver_config[matlab_script]['make_plots'] = False  # Run cells that generate plots?
    driver_config[matlab_script]['plot_settings'] = dict()
    driver_config[matlab_script]['plot_settings']['X'] = 0  # Column to plot output from
    driver_config[matlab_script]['plot_settings']['ylim'] = [5e-7, 50]  # y limits for biomass plots

# configurations that differ from default
driver_config['test_case']['ignore_year_in_forcing'] = True
driver_config['test_locs3']['ignore_year_in_forcing'] = True
driver_config['test_locs3']['FOSI_spinup'] = True
driver_config['test_locs3']['nyears'] = 2
driver_config['test_case']['plot_settings']['ylim'] = [1e-6, 1]
driver_config['test_locs3']['plot_settings']['ylim'] = [1e-6, 100]
for matlab_script in ['FOSI_cesm', 'FOSI_cesm_daily', 'FOSI_spinup']:
    driver_config[matlab_script]['plot_settings']['X'] = 55000  # looks good
    # driver_config[matlab_script]['plot_settings']['X'] = 15633  # large error in small classes
    # driver_config[matlab_script]['plot_settings']['X'] = 11677  # large error in medium classes
    # driver_config[matlab_script]['plot_settings']['X'] = 55000  76989  # large error in benthic

for matlab_script in ['test_case', 'test_locs3']:
    driver_config[matlab_script]['settings_in']['benthic_prey'] = {
        'defaults': {'benthic_efficiency': 0.075, 'carrying_capacity': 0},
        'members': [{'name': 'benthic_prey'}],
    }
    driver_config[matlab_script]['settings_in']['food_web'] = [
        {'predator': 'Sf', 'prey': 'Zoo', 'preference': 1.0},
        {'predator': 'Sp', 'prey': 'Zoo', 'preference': 1.0},
        {'predator': 'Sd', 'prey': 'Zoo', 'preference': 1.0},
        {'predator': 'Mf', 'prey': 'Zoo', 'preference': 0.45},
        {'predator': 'Mf', 'prey': 'Sf', 'preference': 1.0},
        {'predator': 'Mf', 'prey': 'Sp', 'preference': 1.0},
        {'predator': 'Mf', 'prey': 'Sd', 'preference': 1.0},
        {'predator': 'Mp', 'prey': 'Zoo', 'preference': 0.45},
        {'predator': 'Mp', 'prey': 'Sf', 'preference': 1.0},
        {'predator': 'Mp', 'prey': 'Sp', 'preference': 1.0},
        {'predator': 'Mp', 'prey': 'Sd', 'preference': 1.0},
        {'predator': 'Md', 'prey': 'benthic_prey', 'preference': 1.0},
        {'predator': 'Lp', 'prey': 'Mf', 'preference': 0.5},
        {'predator': 'Lp', 'prey': 'Mp', 'preference': 1.0},
        {'predator': 'Lp', 'prey': 'Md', 'preference': 1.0},
        {'predator': 'Ld', 'prey': 'Mf', 'preference': 0.375},
        {'predator': 'Ld', 'prey': 'Mp', 'preference': 0.75},
        {'predator': 'Ld', 'prey': 'Md', 'preference': 1.0},
        {'predator': 'Ld', 'prey': 'benthic_prey', 'preference': 1.0},
    ]

In [5]:
matlab_script = 'test_case'
# matlab_script = 'test_locs3'
# matlab_script = 'FOSI_cesm'

baselines_from_nc = xr.open_dataset(f'../matlab_baselines/{matlab_script}.nc')

if matlab_script not in driver_config:
    raise ValueError(f"unknown matlab_script '{matlab_script}'")

if matlab_script == 'test_case':
    testcase = feisty.config_testcase('tanh_shelf', 'cyclic')
else:
    testcase = feisty.config_from_netcdf(
        './forcing.yaml',
        matlab_script,
        start_date=driver_config[matlab_script]['start_date'],
        ignore_year_in_forcing=driver_config[matlab_script]['ignore_year_in_forcing'],  # True,
        settings_in=driver_config[matlab_script]['settings_in'],
    )

# Set negative forcing values to zero
if driver_config[matlab_script]['force_nonnegative']:
    for var in ['poc_flux_bottom', 'zooC', 'zoo_mort']:
        baselines_from_nc[var].data = np.where(
            baselines_from_nc[var].data > 0, baselines_from_nc[var].data, 0
        )

In [6]:
if driver_config[matlab_script]['compare_forcing']:
    if testcase.ignore_year:
        new_forcing = testcase.forcing.isel(time=slice(1, -1)).transpose('zooplankton', 'time', 'X')
    else:
        new_forcing = testcase.forcing.transpose('zooplankton', 'time', 'X')
    compare_nc(
        baselines_from_nc,
        new_forcing,
    )

    print(
        f"\nMax diff in depth: {np.max(np.abs(baselines_from_nc['dep'].data - testcase.obj.domain_dict['bathymetry'].data))}"
    )

|  | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| T_pelagic (t=0, X=0) | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| T_bottom (t=0, X=0) | 4.2000e+00 | 4.2000e+00 | 0.0000e+00 |
| poc_flux_bottom (t=0, X=0) | 2.7847e-02 | 2.7847e-02 | 0.0000e+00 |
| zooC (t=0, X=0) | 3.3287e+00 | 3.3287e+00 | 0.0000e+00 |
| zoo_mort (t=0, X=0) | 7.7564e-01 | 7.7564e-01 | 0.0000e+00 |

Max diff in depth: 0.0


## Run the model


In [7]:
nsteps = driver_config[matlab_script]['nyears'] * 365
%time testcase.run(nsteps)

CPU times: user 13.5 s, sys: 45.5 ms, total: 13.5 s
Wall time: 15.2 s


In [8]:
compare_nc(baselines_from_nc, testcase.ds["biomass"], nsteps, thres=1e-15)

| group | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf (t=364, X=6) | 1.7697e-05 | 1.7603e-05 | 5.2707e-03 |
| Sp (t=364, X=6) | 9.3653e-06 | 9.3727e-06 | 7.9145e-04 |
| Sd (t=364, X=6) | 9.3653e-06 | 9.3727e-06 | 7.9145e-04 |
| Mf (t=364, X=6) | 4.0856e-04 | 4.0268e-04 | 1.4391e-02 |
| Mp (t=364, X=6) | 3.1926e-04 | 3.1557e-04 | 1.1546e-02 |
| Md (t=364, X=21) | 1.9771e-04 | 1.9534e-04 | 1.1988e-02 |
| Lp (t=364, X=6) | 4.7388e-04 | 4.6868e-04 | 1.0972e-02 |
| Ld (t=364, X=0) | 2.2687e-04 | 2.2398e-04 | 1.2757e-02 |
| benthic_prey (t=364, X=0) | 6.3646e-01 | 6.3395e-01 | 3.9481e-03 |


In [9]:
if driver_config[matlab_script]['make_plots']:
    isel_dict = {"time": range(len(testcase.ds.time))}
    baselines_tmp = baselines_from_nc.isel(isel_dict).assign_coords(
        {'group': testcase.ds.group.data, 'time': testcase.ds.time.data}
    )
    fig, axs = plt.subplots(1, 2, figsize=(15, 4))
    for group in testcase.ds.group.data:
        testcase.ds.biomass.sel(group=group).isel(
            X=driver_config[matlab_script]['plot_settings']['X']
        ).plot(ax=axs[0])
        baselines_tmp.biomass.sel(group=group).isel(
            X=driver_config[matlab_script]['plot_settings']['X']
        ).plot(ax=axs[1])
    for ax in axs:
        ax.set_ylim(driver_config[matlab_script]['plot_settings']['ylim'])
        ax.set_yscale("log")
    axs[0].set_title("python")
    axs[1].set_title("matlab")
    axs[1].set_ylabel("")
    axs[1].set_yticklabels("")
    plt.legend(testcase.ds.group.data, bbox_to_anchor=(1.025, 0.5), loc=6)
    fig.suptitle(f"Comparison at X={driver_config[matlab_script]['plot_settings']['X']}")

In [10]:
if driver_config[matlab_script]['make_plots']:
    fig, axs = plt.subplots(1, 2, figsize=(15, 4))
    for group in testcase.ds.group.data:
        da1 = testcase.ds.biomass.sel(group=group).isel(
            X=driver_config[matlab_script]['plot_settings']['X']
        )
        da2 = baselines_tmp.biomass.sel(group=group).isel(
            X=driver_config[matlab_script]['plot_settings']['X']
        )
        (da1 - da2).plot(ax=axs[0])
        (np.abs(da1 - da2) / da2).plot(ax=axs[1])
    axs[0].set_title("absolute error")
    axs[0].set_ylabel("")
    axs[1].set_title("relative error")
    axs[1].set_ylabel("")
    axs[1].set_yscale("log")
    plt.legend(testcase.ds.group.data, bbox_to_anchor=(1.025, 0.5), loc=6)
    plt.title("Relative Error")
    fig.suptitle(f"Comparison at X={driver_config[matlab_script]['plot_settings']['X']}")

In [11]:
compare_nc(
    baselines_from_nc.isel(X=[driver_config[matlab_script]['plot_settings']['X']]),
    testcase.ds["biomass"].isel(X=[driver_config[matlab_script]['plot_settings']['X']]),
    nsteps,
    thres=1e-15,
)

| group | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf (t=364, X=0) | 1.7696e-05 | 1.7603e-05 | 5.2706e-03 |
| Sp (t=364, X=0) | 9.3653e-06 | 9.3727e-06 | 7.9145e-04 |
| Sd (t=364, X=0) | 9.4481e-06 | 9.4544e-06 | 6.5796e-04 |
| Mf (t=364, X=0) | 4.0856e-04 | 4.0268e-04 | 1.4391e-02 |
| Mp (t=364, X=0) | 3.1926e-04 | 3.1557e-04 | 1.1546e-02 |
| Md (t=364, X=0) | 3.1755e-04 | 3.1387e-04 | 1.1603e-02 |
| Lp (t=364, X=0) | 4.7386e-04 | 4.6866e-04 | 1.0972e-02 |
| Ld (t=364, X=0) | 2.2687e-04 | 2.2398e-04 | 1.2757e-02 |
| benthic_prey (t=364, X=0) | 6.3646e-01 | 6.3395e-01 | 3.9481e-03 |
